In [1]:
using POMDPs
using QuickPOMDPs
using POMDPTools: Uniform, DiscreteUpdater, HistoryRecorder
using Random
using Distributions
using Statistics


# -----------------------------------
# Radar state and observation structs
# -----------------------------------

struct RadarState
    x::Float64
    y::Float64
end

struct RadarObservation
    x_belief::Float64
    y_belief::Float64
end

# --------------------------
# Define support for states
# --------------------------
let
    global RADAR_STATE_SPACE = [RadarState(x, y) for x in -10000:500:10000, y in -10000:500:10000]
end
# Define support for observations
const RADAR_OBS_SPACE = [RadarObservation(x, y) for x in -10000:500:10000, y in -10000:500:10000]

POMDPs.support(::Type{RadarObservation}) = set(RADAR_OBS_SPACE)

POMDPs.support(::RadarState) = Set(RADAR_STATE_SPACE)

POMDPs.pdf(s::RadarState, sp::RadarState) = s == sp ? 1.0 : 0.0


function POMDPs.pdf(o::RadarObservation, op::RadarObservation)
    x_prob = pdf(Normal(op.x_belief, 500.0), o.x_belief)
    y_prob = pdf(Normal(op.y_belief, 500.0), o.y_belief)
    return x_prob * y_prob
end

# -----------------------------
# Helper functions
# -----------------------------

function boresight_polar_2_cartesian(range::Float64, angle_deg::Float64)
    theta_rad = deg2rad(angle_deg)
    return range * cos(theta_rad), range * sin(theta_rad)
end

function snap_to_grid(grid_vals::Vector{Float64}, pos::Float64)
    return grid_vals[argmin(abs.(grid_vals .- pos))]
end


snap_to_grid (generic function with 1 method)

Each step:

Samples a state (always same in this case, since transition is identity and belief collapses).

Chooses an action via the greedy policy

Generates an observation → RadarObservation (with noise)

Computes reward → If observation is accurate (low error), returns 100. If not, returns a penalty based on error and power cost.

Belief is updated → next step → repeat



In [4]:

# -----------------------------
# Define POMDP
# -----------------------------

# Array to log tracking errors
global TRACKING_ERROR = Float64[]
global TRACKING_ERROR_SUCCESS = Float64[]

radar = QuickPOMDP(
    states = RADAR_STATE_SPACE,
    statetype = RadarState,
    obstype = RadarObservation,

    discount = 0.95,

    actions = [(steer, bw, power) for steer in 0:5:355, bw in 5:5:45, power in [100,250,500,1000,1500]],

    transition = function (s, a)
        # simple target movement model (random walk)
    
        dx = randn() * 0.0  # random step in x, ~200m stddev # set to 0.0 (no dynamics)
        dy = randn() * 0.0  # random step in y, ~200m stddev
    
        new_x = clamp(s.x + dx, -10000.0, 10000.0)
        new_y = clamp(s.y + dy, -10000.0, 10000.0)
    
        return RadarState(new_x, new_y)
    end,  # simple random walk dynamics

    observation = function (a, s)
        steer_ang, beamwidth, tx_power = a
        #println("Observation called with state: ", s)

        xpos, ypos = s.x, s.y

        # compute perfect range
        range = sqrt(xpos^2 + ypos^2)

        # SNR model (simple log model)
        snr_db = 10 * log10(tx_power + 1e-3) - 20  # -30 is arbitrary "noise floor" baseline

        # cap SNR to avoid negative infinity
        snr_db = max(snr_db, -20.0)

        # convert SNR to standard deviation: higher SNR -> smaller stddev
        range_noise_std = 1000.0 * 10^(-snr_db / 20)  # higher SNR = lower std
        angle_noise_std = 10.0 * 10^(-snr_db / 20)

        # Add noisef
        measured_range = range + randn() * range_noise_std
        measured_angle = atan(ypos, xpos) * 180 / π + randn() * angle_noise_std

        belief_x, belief_y = boresight_polar_2_cartesian(measured_range, measured_angle)

        return RadarObservation(belief_x, belief_y)
    end,

    reward = function (s, a, sp, o)
        xpos, ypos = s.x, s.y
        tracking_error = sqrt((xpos - o.x_belief)^2 + (ypos - o.y_belief)^2)
        push!(TRACKING_ERROR, tracking_error)  # log success errors
        if tracking_error < 10
            println("Tracking SUCCESS! error = ", tracking_error)
            push!(TRACKING_ERROR_SUCCESS, tracking_error)  # log success errors
            return 100.0
        else
            steer_ang, beamwidth, tx_power = a
            power_cost = tx_power / 100e3
            norm_error = tracking_error / sqrt(2 * (10000^2 + 10000^2))
            return -50 * (power_cost + norm_error)
        end
    end,

    initialstate = Uniform(RADAR_STATE_SPACE),

    isterminal = s -> false
)

# -----------------------------
# Define a simple greedy policy
# -----------------------------

struct SampledGreedyPolicy{M} <: Policy
    pomdp::M
end

POMDPs.action(p::SampledGreedyPolicy, b) = begin
    s = rand(b)  # sample from belief
    best_a = argmax(a -> reward(p.pomdp, s, a, s, observation(p.pomdp, a, s)), actions(p.pomdp))
    return best_a
end

# -----------------------------
# Simple simulator
# -----------------------------

struct MySimulator
    max_steps::Int
end

function POMDPs.simulate(sim::MySimulator, m::POMDP, p::Policy, up::Updater)
    b = initialize_belief(up, initialstate(m))
    s = rand(initialstate(m))

    total_reward = 0.0
    discount_factor = 0.95

    for step in 1:sim.max_steps
        if isterminal(m, s)
            break
        end

        a = action(p, b)

        sp = transition(m, s, a)
        o = observation(m, a, sp)
        r = reward(m, s, a, sp, o)

        println("[Step $step] s=$s → a=$a → o=$o → r=$r → sp=$sp")

        total_reward += discount_factor * r
        discount_factor *= discount(m)

        b = update(up, b, a, o)
        s = sp
    end

    return total_reward
end

# -----------------------------
# Run simulation
# -----------------------------

policy = SampledGreedyPolicy(radar)
updater = DiscreteUpdater(radar)
sim = MySimulator(20)

total_reward = simulate(sim, radar, policy, updater)
println("Total reward: ", total_reward)

if !isempty(TRACKING_ERROR_SUCCESS)
    mean_error = mean(TRACKING_ERROR_SUCCESS)
    std_error = std(TRACKING_ERROR_SUCCESS)
    count_success = length(TRACKING_ERROR_SUCCESS)

    println("\n==== Tracking Success Stats ====")
    println("Number of successes: ", count_success)
    println("Mean error: ", mean_error)
    println("Standard deviation: ", std_error)
else
    println("No tracking successes recorded.")
end

mean_error = mean(TRACKING_ERROR)
std_error = std(TRACKING_ERROR)
count_success = length(TRACKING_ERROR)

println("\n==== Tracking Total Stats ====")
println("Number of successes: ", count_success)
println("Mean error: ", mean_error)
println("Standard deviation: ", std_error)

Tracking SUCCESS! error = 7.089193064535642
[Step 1] s=RadarState(-9500.0, 1500.0) → a=(310, 25, 1000) → o=RadarObservation(-8557.423057003345, 850.7644716360035) → r=-3.3613349864606437 → sp=RadarState(-9500.0, 1500.0)
Tracking SUCCESS! error = 4.874795944976805
Tracking SUCCESS! error = 9.215779492752548
Tracking SUCCESS! error = 6.7129412132859185
[Step 2] s=RadarState(-9500.0, 1500.0) → a=(210, 20, 250) → o=RadarObservation(-10275.12498543814, 594.5668342623505) → r=-3.104752465252126 → sp=RadarState(-9500.0, 1500.0)
[Step 3] s=RadarState(-9500.0, 1500.0) → a=(115, 35, 100) → o=RadarObservation(-9072.122712896205, 3590.6563547667956) → r=-5.384980650472226 → sp=RadarState(-9500.0, 1500.0)
[Step 4] s=RadarState(-9500.0, 1500.0) → a=(140, 45, 100) → o=RadarObservation(-10394.729332652514, -40.59708534318766) → r=-4.503916786153465 → sp=RadarState(-9500.0, 1500.0)
Tracking SUCCESS! error = 6.854490187184329
[Step 5] s=RadarState(-9500.0, 1500.0) → a=(105, 30, 1500) → o=RadarObservatio